## Tutustu K-Means-klusterointiin R:llä ja Tidy data -periaatteilla.

### [**Esiluennon kysely**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/29/)

Tässä oppitunnissa opit luomaan klustereita käyttämällä Tidymodels-pakettia ja muita R-ekosysteemin paketteja (kutsumme niitä ystäviksi 🧑‍🤝‍🧑) sekä aiemmin tuomaasi Nigerian musiikkidataa. Käymme läpi K-Means-klusteroinnin perusteet. Muista, että kuten opit aiemmassa oppitunnissa, klustereiden kanssa työskentelyyn on monia tapoja, ja käyttämäsi menetelmä riippuu datastasi. Kokeilemme K-Means-menetelmää, koska se on yleisin klusterointitekniikka. Aloitetaan!

Termit, joista opit lisää:

-   Silhouette-pisteytys

-   Kyynärpäämenetelmä

-   Inertia

-   Varianssi

### **Johdanto**

[K-Means-klusterointi](https://wikipedia.org/wiki/K-means_clustering) on menetelmä, joka on peräisin signaalinkäsittelyn alalta. Sitä käytetään jakamaan ja ryhmittelemään dataa `k klusteriin` niiden ominaisuuksien samankaltaisuuden perusteella.

Klusterit voidaan visualisoida [Voronoi-diagrammeina](https://wikipedia.org/wiki/Voronoi_diagram), jotka sisältävät pisteen (tai 'siemenen') ja sen vastaavan alueen.

<p >
   <img src="../../images/voronoi.png"
   width="500"/>
   <figcaption>Infografiikka: Jen Looper</figcaption>


K-Means-klusteroinnissa on seuraavat vaiheet:

1.  Data-analyytikko määrittää ensin halutun klustereiden lukumäärän.

2.  Seuraavaksi algoritmi valitsee satunnaisesti K havaintoa datasta, jotka toimivat klustereiden alkuperäisinä keskuksina (eli centroidit).

3.  Tämän jälkeen jokainen jäljellä oleva havainto liitetään lähimpään centroidiin.

4.  Seuraavaksi lasketaan jokaisen klusterin uusi keskiarvo, ja centroidi siirretään keskiarvoon.

5.  Kun keskukset on laskettu uudelleen, jokainen havainto tarkistetaan uudelleen, jotta nähdään, voisiko se olla lähempänä toista klusteria. Kaikki objektit jaetaan uudelleen päivitettyjen klusterikeskiarvojen perusteella. Klusterin jakaminen ja centroidin päivitys toistetaan iteratiivisesti, kunnes klusterien jakaminen ei enää muutu (eli kun konvergenssi saavutetaan). Tyypillisesti algoritmi lopettaa, kun jokainen uusi iteraatio aiheuttaa vain vähäistä centroidien liikettä ja klusterit muuttuvat staattisiksi.

<div>

> Huomaa, että koska alkuperäisten k havaintojen valinta centroidiksi tapahtuu satunnaisesti, voimme saada hieman erilaisia tuloksia joka kerta, kun menetelmää sovelletaan. Tästä syystä useimmat algoritmit käyttävät useita *satunnaisia aloituksia* ja valitsevat iteraation, jolla on pienin WCSS. Siksi on erittäin suositeltavaa aina suorittaa K-Means useilla *nstart*-arvoilla, jotta vältetään *ei-toivottu paikallinen optimi.*

</div>

Tämä lyhyt animaatio, joka käyttää Allison Horstin [taidetta](https://github.com/allisonhorst/stats-illustrations), selittää klusterointiprosessin:

<p >
   <img src="../../images/kmeans.gif"
   width="550"/>
   <figcaption>Taide: @allison_horst</figcaption>



Klusteroinnissa herää perustavanlaatuinen kysymys: mistä tiedät, kuinka moneen klusteriin datasi tulisi jakaa? Yksi K-Means-menetelmän haittapuolista on se, että sinun täytyy määrittää `k`, eli `centroidien` lukumäärä. Onneksi `kyynärpäämenetelmä` auttaa arvioimaan hyvän lähtöarvon `k`:lle. Kokeilet sitä kohta.

### 

**Edellytykset**

Jatkamme suoraan siitä, mihin jäimme [edellisessä oppitunnissa](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb), jossa analysoimme dataa, teimme paljon visualisointeja ja suodattelimme datasta kiinnostavia havaintoja. Muista tarkistaa se!

Tarvitsemme joitakin paketteja tämän moduulin suorittamiseen. Voit asentaa ne seuraavasti: `install.packages(c('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork'))`

Vaihtoehtoisesti alla oleva skripti tarkistaa, onko sinulla tarvittavat paketit tämän moduulin suorittamiseen, ja asentaa ne puolestasi, jos jotkin puuttuvat.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))

pacman::p_load('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork')


Lähdetään vauhdilla liikkeelle!

## 1. Tanssi datan kanssa: Rajaa kolmeen suosituimpaan musiikkigenreen

Tämä on kertaus siitä, mitä teimme edellisessä oppitunnissa. Pilkotaan ja analysoidaan hieman dataa!


In [ ]:
# Load the core tidyverse and make it available in your current R session
library(tidyverse)

# Import the data into a tibble
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/5-Clustering/data/nigerian-songs.csv", show_col_types = FALSE)

# Narrow down to top 3 popular genres
nigerian_songs <- df %>% 
  # Concentrate on top 3 genres
  filter(artist_top_genre %in% c("afro dancehall", "afropop","nigerian pop")) %>% 
  # Remove unclassified observations
  filter(popularity != 0)



# Visualize popular genres using bar plots
theme_set(theme_light())
nigerian_songs %>%
  count(artist_top_genre) %>%
  ggplot(mapping = aes(x = artist_top_genre, y = n,
                       fill = artist_top_genre)) +
  geom_col(alpha = 0.8) +
  paletteer::scale_fill_paletteer_d("ggsci::category10_d3") +
  ggtitle("Top genres") +
  theme(plot.title = element_text(hjust = 0.5))


🤩 Se meni hyvin!

## 2. Lisää datan tutkimista.

Kuinka puhdasta tämä data on? Tarkistetaan poikkeamat käyttämällä laatikkokaavioita. Keskitymme numeerisiin sarakkeisiin, joissa on vähemmän poikkeamia (vaikka voisitkin siivota poikkeamat pois). Laatikkokaaviot voivat näyttää datan vaihteluvälin ja auttaa valitsemaan, mitä sarakkeita käyttää. Huomaa, että laatikkokaaviot eivät näytä varianssia, joka on tärkeä osa hyvää klusteroitavaa dataa. Katso [tämä keskustelu](https://stats.stackexchange.com/questions/91536/deduce-variance-from-boxplot) lisälukemista varten.

[Laatikkokaaviot](https://en.wikipedia.org/wiki/Box_plot) käytetään numeerisen datan jakauman graafiseen esittämiseen, joten aloitetaan *valitsemalla* kaikki numeeriset sarakkeet yhdessä suosittujen musiikkigenrejen kanssa.


In [ ]:
# Select top genre column and all other numeric columns
df_numeric <- nigerian_songs %>% 
  select(artist_top_genre, where(is.numeric)) 

# Display the data
df_numeric %>% 
  slice_head(n = 5)


Katso, kuinka valintatyökalu `where` tekee tämän helpoksi 💁? Tutustu muihin vastaaviin toimintoihin [täällä](https://tidyselect.r-lib.org/).

Koska aiomme tehdä laatikkokaavion jokaiselle numeeriselle ominaisuudelle ja haluamme välttää silmukoiden käyttöä, muotoillaan datamme *pidempään* muotoon, joka mahdollistaa `facets`-ominaisuuden hyödyntämisen - alikuvioita, jotka näyttävät kunkin datan osajoukon.


In [ ]:
# Pivot data from wide to long
df_numeric_long <- df_numeric %>% 
  pivot_longer(!artist_top_genre, names_to = "feature_names", values_to = "values") 

# Print out data
df_numeric_long %>% 
  slice_head(n = 15)


Paljon pidempään! Nyt on aika käyttää `ggplotseja`! Joten mitä `geomia` käytämme?


In [ ]:
# Make a box plot
df_numeric_long %>% 
  ggplot(mapping = aes(x = feature_names, y = values, fill = feature_names)) +
  geom_boxplot() +
  facet_wrap(~ feature_names, ncol = 4, scales = "free") +
  theme(legend.position = "none")


Nyt voimme nähdä, että nämä tiedot ovat hieman meluisia: tarkastelemalla kutakin saraketta laatikkokaaviona, voit havaita poikkeavia arvoja. Voisit käydä datasetin läpi ja poistaa nämä poikkeamat, mutta se tekisi datasta melko vähäistä.

Tällä kertaa valitaan sarakkeet, joita käytämme ryhmittelyharjoituksessa. Valitaan numeeriset sarakkeet, joilla on samankaltaiset vaihteluvälit. Voisimme koodata `artist_top_genre` numeeriseksi, mutta jätämme sen pois tällä kertaa.


In [ ]:
# Select variables with similar ranges
df_numeric_select <- df_numeric %>% 
  select(popularity, danceability, acousticness, loudness, energy) 

# Normalize data
# df_numeric_select <- scale(df_numeric_select)


## 3. K-means-klusteroinnin laskeminen R:ssä

Voimme laskea k-means-klusteroinnin R:ssä sisäänrakennetulla `kmeans`-funktiolla, katso `help("kmeans()")`. `kmeans()`-funktio hyväksyy ensisijaisena argumenttinaan dataframen, jossa kaikki sarakkeet ovat numeerisia.

Ensimmäinen vaihe k-means-klusterointia käytettäessä on määrittää klustereiden (k) lukumäärä, jotka muodostuvat lopullisessa ratkaisussa. Tiedämme, että aineistosta on eroteltu kolme musiikkigenreä, joten kokeillaan kolmea:


In [ ]:
set.seed(2056)
# Kmeans clustering for 3 clusters
kclust <- kmeans(
  df_numeric_select,
  # Specify the number of clusters
  centers = 3,
  # How many random initial configurations
  nstart = 25
)

# Display clustering object
kclust


Kmeans-objekti sisältää useita tietoja, jotka on hyvin selitetty kohdassa `help("kmeans()")`. Keskitytään nyt muutamaan niistä. Näemme, että data on ryhmitelty kolmeen klusteriin, joiden koot ovat 65, 110 ja 111. Tuloste sisältää myös klustereiden keskukset (keskiarvot) näille kolmelle ryhmälle viiden muuttujan osalta.

Klusterointivektori on klusterin määrittely jokaiselle havainnolle. Käytetään `augment`-funktiota, jotta klusterin määrittely lisätään alkuperäiseen datajoukkoon.


In [ ]:
# Add predicted cluster assignment to data set
augment(kclust, df_numeric_select) %>% 
  relocate(.cluster) %>% 
  slice_head(n = 10)


Loistavaa, olemme juuri jakaneet tietojoukon kolmeen ryhmään. Mutta kuinka hyvä onkaan ryhmittelymme 🤷? Katsotaanpa `Silhouette-pistemäärää`.

### **Silhouette-pistemäärä**

[Silhouette-analyysi](https://en.wikipedia.org/wiki/Silhouette_(clustering)) voidaan käyttää tutkimaan etäisyyttä muodostettujen ryhmien välillä. Tämä pistemäärä vaihtelee välillä -1 ja 1, ja jos pistemäärä on lähellä 1, ryhmä on tiivis ja hyvin erillään muista ryhmistä. Arvo lähellä 0 edustaa päällekkäisiä ryhmiä, joissa näytteet ovat hyvin lähellä naapuriryhmien päätösrajaa. [Lähde](https://dzone.com/articles/kmeans-silhouette-score-explained-with-python-exam).

Keskimääräinen silhouette-menetelmä laskee havaintojen keskimääräisen silhouette-pistemäärän eri *k*-arvoille. Korkea keskimääräinen silhouette-pistemäärä osoittaa hyvää ryhmittelyä.

`silhouette`-funktiota klusteripaketissa käytetään laskemaan keskimääräinen silhouette-leveys.

> Silhouette voidaan laskea millä tahansa [etäisyys](https://en.wikipedia.org/wiki/Distance "Distance")-metriikalla, kuten [Euklidinen etäisyys](https://en.wikipedia.org/wiki/Euclidean_distance "Euclidean distance") tai [Manhattanin etäisyys](https://en.wikipedia.org/wiki/Manhattan_distance "Manhattan distance"), joita käsittelimme [edellisessä oppitunnissa](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb).


In [ ]:
# Load cluster package
library(cluster)

# Compute average silhouette score
ss <- silhouette(kclust$cluster,
                 # Compute euclidean distance
                 dist = dist(df_numeric_select))
mean(ss[, 3])


Pisteemme on **0.549**, eli aivan keskivaiheilla. Tämä osoittaa, että datamme ei ole erityisen hyvin soveltuvaa tämän tyyppiseen klusterointiin. Katsotaan, voimmeko vahvistaa tätä epäilystä visuaalisesti. [factoextra-paketti](https://rpkgs.datanovia.com/factoextra/index.html) tarjoaa toimintoja (`fviz_cluster()`) klusteroinnin visualisointiin.


In [ ]:
library(factoextra)

# Visualize clustering results
fviz_cluster(kclust, df_numeric_select)


Klustereiden päällekkäisyys osoittaa, että datamme ei ole erityisen hyvin soveltuva tämän tyyppiseen klusterointiin, mutta jatketaan silti.

## 4. Optimaalisten klustereiden määrittäminen

K-Means-klusteroinnissa usein herää perustavanlaatuinen kysymys - ilman tunnettuja luokkamerkintöjä, miten tiedät, mihin määrään klustereita data tulisi jakaa?

Yksi tapa selvittää tämä on käyttää datanäytettä `luodaksesi sarjan klusterointimalleja`, joissa klustereiden määrä kasvaa asteittain (esim. 1-10), ja arvioida klusterointimittareita, kuten **Silhouette-pisteytys.**

Määritetään optimaalinen klustereiden määrä laskemalla klusterointialgoritmi eri *k*-arvoille ja arvioimalla **Within Cluster Sum of Squares** (WCSS). Kokonais-WCSS mittaa klusteroinnin tiiviyttä, ja haluamme sen olevan mahdollisimman pieni, sillä pienemmät arvot tarkoittavat, että datapisteet ovat lähempänä toisiaan.

Tutkitaan, miten eri `k`-valinnat, välillä 1–10, vaikuttavat tähän klusterointiin.


In [ ]:
# Create a series of clustering models
kclusts <- tibble(k = 1:10) %>% 
  # Perform kmeans clustering for 1,2,3 ... ,10 clusters
  mutate(model = map(k, ~ kmeans(df_numeric_select, centers = .x, nstart = 25)),
  # Farm out clustering metrics eg WCSS
         glanced = map(model, ~ glance(.x))) %>% 
  unnest(cols = glanced)
  

# View clustering rsulsts
kclusts


Nyt kun meillä on kokonaisklusterin sisäinen neliösumma (tot.withinss) jokaiselle klusterointialgoritmille keskuksella *k*, käytämme [kyynärpäämenetelmää](https://en.wikipedia.org/wiki/Elbow_method_(clustering)) löytääksemme optimaalisen klustereiden määrän. Menetelmä koostuu WCSS:n piirtämisestä klustereiden määrän funktiona ja [käyrän kyynärpään](https://en.wikipedia.org/wiki/Elbow_of_the_curve "Elbow of the curve") valitsemisesta käytettäväksi klustereiden määräksi.


In [ ]:
set.seed(2056)
# Use elbow method to determine optimum number of clusters
kclusts %>% 
  ggplot(mapping = aes(x = k, y = tot.withinss)) +
  geom_line(size = 1.2, alpha = 0.8, color = "#FF7F0EFF") +
  geom_point(size = 2, color = "#FF7F0EFF")


Kuvaaja näyttää merkittävän vähennyksen WCSS:ssä (eli suurempi *tiiviys*), kun klustereiden määrä kasvaa yhdestä kahteen, ja edelleen huomattavan vähennyksen kahdesta kolmeen klusteriin. Tämän jälkeen vähennys on vähemmän merkittävä, mikä johtaa `kyynärpään` 💪 kaaviossa noin kolmen klusterin kohdalla. Tämä on hyvä merkki siitä, että datan pisteet muodostavat kaksi tai kolme kohtuullisen hyvin erillistä klusteria.

Voimme nyt jatkaa ja ottaa käyttöön klusterointimallin, jossa `k = 3`:

> `pull()`: käytetään yhden sarakkeen poimimiseen
>
> `pluck()`: käytetään tietorakenteiden, kuten listojen, indeksointiin


In [ ]:
# Extract k = 3 clustering
final_kmeans <- kclusts %>% 
  filter(k == 3) %>% 
  pull(model) %>% 
  pluck(1)


final_kmeans


Hienoa! Visualisoidaan nyt saadut klusterit. Kiinnostaisiko hieman interaktiivisuutta käyttämällä `plotly`-kirjastoa?


In [ ]:
# Add predicted cluster assignment to data set
results <-  augment(final_kmeans, df_numeric_select) %>% 
  bind_cols(df_numeric %>% select(artist_top_genre)) 

# Plot cluster assignments
clust_plt <- results %>% 
  ggplot(mapping = aes(x = popularity, y = danceability, color = .cluster, shape = artist_top_genre)) +
  geom_point(size = 2, alpha = 0.8) +
  paletteer::scale_color_paletteer_d("ggthemes::Tableau_10")

ggplotly(clust_plt)


Ehkä olisimme odottaneet, että jokaisella klusterilla (esitetty eri väreillä) olisi selkeästi erottuvat genret (esitetty eri muodoilla).

Katsotaanpa mallin tarkkuutta.


In [ ]:
# Assign genres to predefined integers
label_count <- results %>% 
  group_by(artist_top_genre) %>% 
  mutate(id = cur_group_id()) %>% 
  ungroup() %>% 
  summarise(correct_labels = sum(.cluster == id))


# Print results  
cat("Result:", label_count$correct_labels, "out of", nrow(results), "samples were correctly labeled.")

cat("\nAccuracy score:", label_count$correct_labels/nrow(results))


Tämän mallin tarkkuus ei ole huono, mutta ei myöskään erinomainen. On mahdollista, että data ei sovellu hyvin K-Means-klusterointiin. Tämä data on liian epätasapainoista, liian vähän korreloitunutta ja sarakkeiden arvojen välillä on liikaa vaihtelua, jotta klusterointi toimisi hyvin. Itse asiassa muodostuvat klusterit ovat todennäköisesti vahvasti vinoutuneita tai vaikuttavat suuresti niistä kolmesta genre-kategoriasta, jotka määrittelimme aiemmin.

Siitä huolimatta, tämä oli varsin opettavainen prosessi!

Scikit-learnin dokumentaatiosta voit nähdä, että tällaisella mallilla, jossa klusterit eivät ole kovin selkeästi rajattuja, on "varianssi"-ongelma:

<p >
   <img src="../../images/problems.png"
   width="500"/>
   <figcaption>Infografiikka Scikit-learnilta</figcaption>



## **Varianssi**

Varianssi määritellään "keskiarvosta laskettujen neliöityjen poikkeamien keskiarvona" [lähde](https://www.mathsisfun.com/data/standard-deviation.html). Tämän klusterointiongelman yhteydessä se viittaa dataan, jossa datasetin luvut poikkeavat keskiarvosta hieman liikaa.

✅ Tämä on loistava hetki miettiä kaikkia tapoja, joilla voisit korjata tämän ongelman. Voisiko dataa muokata vielä hieman? Käyttää eri sarakkeita? Käyttää eri algoritmia? Vinkki: Kokeile [skaalata dataasi](https://www.mygreatlearning.com/blog/learning-data-science-with-k-means-clustering/) normalisoidaksesi sen ja testaa muita sarakkeita.

> Kokeile tätä '[varianssilaskuria](https://www.calculatorsoup.com/calculators/statistics/variance-calculator.php)' ymmärtääksesi konseptia hieman paremmin.

------------------------------------------------------------------------

## **🚀Haaste**

Käytä aikaa tämän notebookin parissa ja säädä parametreja. Voitko parantaa mallin tarkkuutta puhdistamalla dataa enemmän (esimerkiksi poistamalla poikkeavia arvoja)? Voit käyttää painotuksia antaaksesi enemmän painoarvoa tietyille datasampleille. Mitä muuta voisit tehdä luodaksesi parempia klustereita?

Vinkki: Kokeile skaalata dataasi. Notebookissa on kommentoitua koodia, joka lisää standardisoinnin, jotta datan sarakkeet muistuttaisivat toisiaan enemmän vaihteluvälin suhteen. Huomaat, että vaikka siluettipisteet laskevat, kyynärpääkäyrän "mutka" tasoittuu. Tämä johtuu siitä, että skaalaamaton data antaa vähemmän vaihtelevalle datalle enemmän painoarvoa. Lue lisää tästä ongelmasta [täältä](https://stats.stackexchange.com/questions/21222/are-mean-normalization-and-feature-scaling-needed-for-k-means-clustering/21226#21226).

## [**Luentojälkeinen kysely**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/30/)

## **Kertaus ja itseopiskelu**

-   Tutustu K-Means-simulaattoriin [kuten tähän](https://user.ceng.metu.edu.tr/~akifakkus/courses/ceng574/k-means/). Voit käyttää tätä työkalua visualisoidaksesi esimerkkidatapisteitä ja määrittääksesi niiden keskipisteet. Voit muokata datan satunnaisuutta, klustereiden määrää ja keskipisteiden määrää. Auttaako tämä sinua hahmottamaan, miten data voidaan ryhmitellä?

-   Tutustu myös [tähän K-Means-materiaaliin](https://stanford.edu/~cpiech/cs221/handouts/kmeans.html) Stanfordilta.

Haluatko kokeilla juuri oppimiasi klusterointitaitoja datasetteihin, jotka soveltuvat hyvin K-Means-klusterointiin? Katso:

-   [Klusterointimallien koulutus ja arviointi](https://rpubs.com/eR_ic/clustering) käyttäen Tidymodelsia ja muita työkaluja

-   [K-means-klusterianalyysi](https://uc-r.github.io/kmeans_clustering), UC Business Analytics R Programming Guide

- [K-means-klusterointi tidy data -periaatteilla](https://www.tidymodels.org/learn/statistics/k-means/)

## **Tehtävä**

[Kokeile erilaisia klusterointimenetelmiä](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/2-K-Means/assignment.md)

## KIITOKSET:

[Jen Looper](https://www.twitter.com/jenlooper) alkuperäisen Python-version luomisesta tästä moduulista ♥️

[`Allison Horst`](https://twitter.com/allison_horst/) upeiden kuvitusten luomisesta, jotka tekevät R:stä lähestyttävämmän ja kiinnostavamman. Löydät lisää kuvituksia hänen [galleriastaan](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

Iloisia oppimishetkiä,

[Eric](https://twitter.com/ericntay), Gold Microsoft Learn Student Ambassador.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="500"/>
   <figcaption>Kuvitus: @allison_horst</figcaption>



---

**Vastuuvapauslauseke**:  
Tämä asiakirja on käännetty käyttämällä tekoälypohjaista käännöspalvelua [Co-op Translator](https://github.com/Azure/co-op-translator). Vaikka pyrimme tarkkuuteen, huomioithan, että automaattiset käännökset voivat sisältää virheitä tai epätarkkuuksia. Alkuperäistä asiakirjaa sen alkuperäisellä kielellä tulee pitää ensisijaisena lähteenä. Kriittisen tiedon osalta suositellaan ammattimaista ihmiskääntämistä. Emme ole vastuussa väärinkäsityksistä tai virhetulkinnoista, jotka johtuvat tämän käännöksen käytöstä.
